In [2]:
!pip install pyvirtualdisplay Pillow


In [3]:
import os
import torch
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import gymnasium as gym
from tqdm import tqdm
from dotenv import load_dotenv
from gymnasium import spaces
import matplotlib.patches as patches

SUMO_HOME = 'C:\\Program Files (x86)\\Eclipse\\Sumo'

os.environ['SUMO_HOME'] = SUMO_HOME

# Print the SUMO_HOME environment variable to verify
print("SUMO_HOME is set to:", os.environ.get('SUMO_HOME'))

# Check if the SUMO_HOME path exists
print("Does SUMO_HOME exist?", os.path.exists(os.environ.get('SUMO_HOME')))


# Define file paths
net_file = 'C:/Users/ASUS/Sumo/2024-11-03-18-23-47/osm.net.xml/osm.net.xml'
route_file = 'C:/Users/ASUS/Sumo/2024-11-03-18-23-47/osm.passenger.rou.xml'
out_csv_name = 'C:/Users/ASUS/Sumo/2024-11-03-18-23-47/osm.passenger.csv'
config_file = 'C:/Users/ASUS/Sumo/2024-11-03-18-23-47/osm.sumocfg'


SUMO_HOME is set to: C:\Program Files (x86)\Eclipse\Sumo
Does SUMO_HOME exist? True


In [1]:
import gymnasium as gym
import sumo_rl
import os
import traci

class Environment:
    def __init__(self, env_name, route_file, net_file, out_csv_name, render_mode='human'):
        self.env_name = env_name
        self.use_gui = True if render_mode == 'human' else False
        self.route_file = route_file
        self.net_file = net_file
        self.out_csv_name = out_csv_name
        self.env = gym.make(
                        env_name,
                        net_file=net_file,
                        route_file=route_file,
                        out_csv_name=out_csv_name,
                        use_gui=self.use_gui,
                        num_seconds=100000
                    )
     
        self.state, _ = self.env.reset()
        self.done = False
        self.observation_space = self.env.observation_space.shape[0]
        self.action_space = self.env.action_space
        self.traffic_signals = self.env.traffic_signals
    
    def reset(self):
        self.state, _ = self.env.reset()
        self.done = False
        return self.state
    
    def custom_reward(self, traffic_signal, reward_type='average_speed', reward_method='simple'):
        if reward_method == 'simple':
            match reward_type:
                case 'average_speed':
                    return traffic_signal.get_avgerage_speed()
                case 'waiting_time':
                    return -1 * traffic_signal.get_total_waiting_time()
                case 'emissions':
                    return -1* traffic_signal.get_emission_co2()
                case 'throughput':
                    return traffic_signal.get_throughput()

        else:
            # Weighted sum of the metrics
            reward = 0
            if weights is None:
                weights = {
                    'average_speed': 0.4,
                    'waiting_time': 0.3,
                    'emissions': 0.2,
                    'throughput': 0.1
                }

            # Calculate individual rewards
            average_speed = traffic_signal.get_average_speed()
            waiting_time = -1* traffic_signal._diff_waiting_time_reward()
            total_queue = -1 * traffic_signal.get_total_queued()
            congesion = traffic_signal.get_pressure()

            print(average_speed, waiting_time, total_queue, congesion)
            weighted_reward = (
                weights['average_speed'] * average_speed +
                weights['waiting_time'] * waiting_time +
                weights['emissions'] * total_queue +
                weights['throughput'] * congesion
            )

            return weighted_reward
            
            
    def step(self, action):
        next_state, _, terminated, truncated, info = self.env.step(action)

        traffic_signal = list(self.traffic_signals.values())[0]
        print(traffic_signal.get_average_speed(), traffic_signal.get_total_queued(), traffic_signal._diff_waiting_time_reward(), traffic_signal.get_pressure())
        reward = self.custom_reward(traffic_signal, reward_type='weighted')

        self.state = next_state
        self.done = terminated
        return next_state, reward, self.done or truncated
    
    def render(self):
        self.env.render()
    
    def close(self):
        try:
            self.env.close()
            if traci.isLoaded():
                traci.close()
            print("Env and Traci closed successfully.")
        except Exception as e:
            print("Error while closing the environment:", e)
    
    def get_state(self):
        return self.state


## Running Environment (Simple Intersection)

In [4]:
# Define paths to the network and route files (Buffalo Road Intersection)

# net_file = 'C:/Users/ASUS/Sumo/2024-11-03-18-23-47/osm.net.xml/osm.net.xml'
# route_file = 'C:/Users/ASUS/Sumo/2024-11-03-18-23-47/osm.passenger.rou.xml'
# out_csv_name = 'C:/Users/ASUS/Sumo/2024-11-03-18-23-47/osm.passenger.csv'


# Using Custom simple Traffic Intersection
nets_dir = 'nets'

file_name = 'single_intersection_simple'

nets_file = os.path.join(nets_dir, f'{file_name}.net.xml')
routes_file = os.path.join(nets_dir, f'{file_name}.rou.xml')

file_exists = lambda file_path: os.path.exists(file_path)

if not file_exists(nets_file):
    raise FileNotFoundError(f"Net file not found: {net_file}")
if not file_exists(routes_file):
    raise FileNotFoundError(f"Route file not found: {route_file}")


# Instantiate the environment
sumo_env = Environment('sumo-rl-v0', net_file=nets_file, route_file=routes_file, out_csv_name=out_csv_name, render_mode='human')

print("Observation Space:", sumo_env.observation_space)
print("Action Space:", sumo_env.action_space)
print("Initial State:", sumo_env.state)

max_steps = 1000

try:
    print("action space", sumo_env.action_space)
    for step in range(max_steps):
        sumo_env.render()
        action = sumo_env.env.action_space.sample()
        next_state, reward, done = sumo_env.step(action)
        print("Action ", action, " Reward ", reward, " Done ", done, "observation", next_state)
        
        if done:
            sumo_env.reset()
            break

except KeyboardInterrupt:
    print("Keyboard interrupt detected. Closing the environment.")

except Exception as e:
    print("Error occurred:", e)

finally:
    sumo_env.close()

 Retrying in 1 seconds


c:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\site-packages\gymnasium\core.py:311: UserWarning: WARN: env.traffic_signals to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.traffic_signals` for environment variables or `env.get_wrapper_attr('traffic_signals')` that will search the reminding wrappers.
  logger.warn(
c:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\site-packages\gymnasium\utils\passive_env_checker.py:335: UserWarning: WARN: No render fps was declared in the environment (env.metadata['render_fps'] is None or not defined), rendering may occur at inconsistent fps.
  logger.warn(


Observation Space: 11
Action Space: Discrete(2)
Initial State: [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
action space Discrete(2)
0.9540128747125406 0 0.0 -7
Action  1  Reward  None  Done  False observation [1.         0.         0.         0.05048805 0.05048805 0.15850651
 0.10567101 0.         0.         0.         0.        ]
0.8855570682162038 0 0.0 -8
Action  1  Reward  None  Done  False observation [1.         0.         1.         0.05048805 0.05048805 0.15850651
 0.15850651 0.         0.         0.         0.        ]
0.6432808450936892 1 0.0 -10
Action  0  Reward  None  Done  False observation [1.         0.         1.         0.05048805 0.05048805 0.26417753
 0.21134202 0.         0.         0.05283551 0.        ]
0.5465994873298571 5 0.0 -12
Action  0  Reward  None  Done  False observation [1.         0.         1.         0.1009761  0.05048805 0.31701303
 0.26417753 0.         0.         0.15850651 0.10567101]
0.4642303606975259 0 0.0 -10
Action  1  Reward  None  Done  False obser

In [1]:
import gymnasium as gym
from sumo_rl import SumoEnvironment
import os

# Print SUMO_HOME for confirmation
print("SUMO_HOME is set to:", os.environ['SUMO_HOME'])

# Initialize SUMO Gym environment
env = gym.make(
    'sumo-rl-v0',
    net_file=net_file,
    route_file=route_file,
    out_csv_name=out_csv_name,
    use_gui=True,
    num_seconds=100000
)

# Reset the environment and start simulation
obs, info = env.reset()
done = False

print(env.action_space)

while not done:
    next_obs, reward, terminated, truncated, info = env.step(env.action_space.sample())
    print("Reward:", reward)
    print("Observation:", next_obs)
    done = terminated or truncated


KeyboardInterrupt: 

In [9]:
import sumolib

# Load the network file
net = sumolib.net.readNet(nets_file)  # Replace with your network file

# Get all edge IDs
edge_ids = [edge.getID() for edge in net.getEdges()]
print("Edge IDs:", edge_ids)

# List all traffic light IDs
tls_ids = [tls.getID() for tls in net.getTrafficLights()]
print("Traffic Light IDs:", tls_ids)

# # Get all vehicle IDs in the current step
# vehicle_ids = traci.vehicle.getIDList()
# print("Vehicle IDs:", vehicle_ids)




Edge IDs: ['n_t', 't_e', 't_s', 'w_t']
Traffic Light IDs: ['t']


In [6]:
import traci
import sumolib
import sys
import time

try:
    # Define paths to SUMO binary, network file, and route file
    sumo_binary = "sumo-gui"  # Use "sumo" for non-GUI mode
    # Command to start SUMO with TraCI connection
    sumo_cmd = [sumo_binary, "-n", nets_file, "-r", routes_file, "--start"]
    
    # Attempt to start SUMO with the specified command
    try:
        traci.start(sumo_cmd, label="sim1")
    except traci.exceptions.TraCIException as e:
        print(f"Failed to start TraCI: {e}")
        sys.exit(1)

    print("TraCI connected successfully.")
    
    # Main simulation loop with safe TraCI operations
    for step in range(1000):
        try:
            traci.simulationStep()  # Advance simulation by one step
            
            # Check vehicle count on a specific edge
            edge_id = "t_e"  # Replace with actual edge ID
            try:
                vehicle_count = traci.edge.getLastStepVehicleNumber(edge_id)
                print(f"Step {step}, Vehicle count on {edge_id}: {vehicle_count}")
            except traci.exceptions.TraCIException as e:
                print(f"Error retrieving vehicle count for edge {edge_id}: {e}")
            
            # Get traffic light phase and queue length
            tls_id = "t"  # Replace with actual traffic light ID
            try:
                phase = traci.trafficlight.getPhase(tls_id)
                print(f"Traffic light {tls_id} phase: {phase}")
            except traci.exceptions.TraCIException as e:
                print(f"Error retrieving traffic light phase for {tls_id}: {e}")
            
            # Pause to inspect values for each step
            # input("Press Enter to continue to the next step...")
            time.sleep(0.5)
            
            if step >= max_steps -1:
                break            

        except KeyboardInterrupt:
            print("\nSimulation interrupted by user.")
            break  # Safely exit the loop if user interrupts

except (FileNotFoundError, traci.exceptions.TraCIException) as e:
    print(f"Error in setup or simulation: {e}")

except Exception as e:
    print(f"Unexpected error occurred: {e}")

finally:
    # Ensure that TraCI closes safely even if an error occurs
    try:
        if traci.isLoaded():
            traci.close()
            print("TraCI connection closed safely.")
    except traci.exceptions.TraCIException as e:
        print(f"Error while closing TraCI: {e}")


 Retrying in 1 seconds
TraCI connected successfully.
Step 0, Vehicle count on t_e: 0
Traffic light t phase: 0
Step 1, Vehicle count on t_e: 0
Traffic light t phase: 0
Step 2, Vehicle count on t_e: 0
Traffic light t phase: 0
Step 3, Vehicle count on t_e: 0
Traffic light t phase: 0
Step 4, Vehicle count on t_e: 0
Traffic light t phase: 0
Step 5, Vehicle count on t_e: 0
Traffic light t phase: 0
Step 6, Vehicle count on t_e: 0
Traffic light t phase: 0
Step 7, Vehicle count on t_e: 0
Traffic light t phase: 0
Step 8, Vehicle count on t_e: 0
Traffic light t phase: 0
Step 9, Vehicle count on t_e: 0
Traffic light t phase: 0
Step 10, Vehicle count on t_e: 0
Traffic light t phase: 0
Step 11, Vehicle count on t_e: 0
Traffic light t phase: 0
Step 12, Vehicle count on t_e: 0
Traffic light t phase: 0
Step 13, Vehicle count on t_e: 0
Traffic light t phase: 0
Step 14, Vehicle count on t_e: 0
Traffic light t phase: 0
Step 15, Vehicle count on t_e: 0
Traffic light t phase: 0
Step 16, Vehicle count on t_e

FatalTraCIError: Connection closed by SUMO.